In [2]:
import numpy as np 
import pandas as pd
import os

In [6]:
!pip install kagglehub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 kB 1.1 MB/s eta 0:00:00a 0:00:01


In [8]:
import kagglehub
path = kagglehub.dataset_download("uciml/pima-indians-diabetes-database")
print("Path to dataset files:", path)

100%|██████████████████████████████████████| 8.91k/8.91k [00:00<00:00, 5.97MB/s]

Extracting files...
Path to dataset files: /Users/khushimodi/.cache/kagglehub/datasets/uciml/pima-indians-diabetes-database/versions/1


In [12]:
df=pd.read_csv("diabetes.csv")
df.head(5)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [14]:
#create a column BMI_category
df['BMI_category'] = np.where(df['BMI'] < 18.5, 'Underweight',
                              np.where(df['BMI'] < 24.9, 'Normal', 
                                       np.where(df['BMI'] < 29.9, 'Overweight',
                                                np.where(df['BMI'] >= 30,'Obese','NA'))))

In [16]:
#check na values in bmi_category
df['BMI_category'].isna().sum()

0

In [18]:
#check na values in dataset
df.isna().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
BMI_category                0
dtype: int64

In [20]:
X=df.drop('Outcome',axis=1)
X.head()
y=df['Outcome']

In [22]:
#80/20 split for train and validation
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

In [23]:
#Apply Standard Scaler on the numeric features. Fit and transform on train and only transform on val.
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
numeric_features = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']
X_train[numeric_features] = sc.fit_transform(X_train[numeric_features])
X_val[numeric_features] = sc.transform(X_val[numeric_features])

In [24]:
X_train.nunique()

Pregnancies                  17
Glucose                     128
BloodPressure                47
SkinThickness                51
Insulin                     166
BMI                         230
DiabetesPedigreeFunction    440
Age                          52
BMI_category                  5
dtype: int64

In [28]:
#Apple One-hot Encoding to the categorical features. Fit and transform on train and only transform on val.
X_train = pd.get_dummies(X_train, columns=['BMI_category'], drop_first=True)
X_val = pd.get_dummies(X_val, columns=['BMI_category'], drop_first=True)
dummy_columns = X_train.columns
X_val = X_val.reindex(columns=X_train.columns, fill_value=0)

In [30]:
#Build a KNN classifier. Experiment with different values of k (3, 5, 7) and select the value with the highest f1 score.
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
k_values = [3, 5, 7]
f1_scores = []
for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_val)
    f1_scores.append(f1_score(y_val, y_pred))
best_k = k_values[np.argmax(f1_scores)]
print("Best k value:", best_k)

Best k value: 5


In [32]:
#Build a Decision Tree classifier. Experiment with different values of max_depth (3, 5, 7) and choose the value with the highest f1 score.
from sklearn.tree import DecisionTreeClassifier
max_depth_values = [3, 5, 7]
f1_scores = []
for max_depth in max_depth_values:
    dt = DecisionTreeClassifier(max_depth=max_depth, random_state=0)
    dt.fit(X_train, y_train)
    y_pred = dt.predict(X_val)
    f1_scores.append(f1_score(y_val, y_pred))
best_max_depth = max_depth_values[np.argmax(f1_scores)]
print("Best max_depth value:", best_max_depth)

Best max_depth value: 7


In [34]:
import joblib
knn=KNeighborsClassifier(n_neighbors=best_k)
knn.fit(X_train, y_train)
joblib.dump(sc, 'scaler.pkl')
joblib.dump(dummy_columns, 'dummy_columns.pkl')
joblib.dump(knn, 'knn_model.pkl')

['knn_model.pkl']

In [36]:
dt=DecisionTreeClassifier(max_depth=best_max_depth, random_state=0)
dt.fit(X_train, y_train)
joblib.dump(dt, 'dt_model.pkl')

['dt_model.pkl']

In [38]:
X_val_sample = X_val.sample(5, random_state=0)
y_val_sample = y_val.loc[X_val_sample.index]
X_val_sample.to_csv('X_val_sample.csv', index=False)
y_val_sample.to_csv('y_val_sample.csv', index=False)

In [40]:
import joblib
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# Load saved models and transformations
scaler = joblib.load('scaler.pkl')
encoder = joblib.load('dummy_columns.pkl')
knn = joblib.load('knn_model.pkl')
dt = joblib.load('dt_model.pkl')
# Load the sample data
X_val_sample = pd.read_csv('X_val_sample.csv')
y_val_sample = pd.read_csv('y_val_sample.csv')
# Reindex to match the training set structure (including one-hot encoded BMI categories)
X_val_sample = X_val_sample.reindex(columns=X_train.columns, fill_value=0)
# Apply scaling on numerical columns (excluding the one-hot encoded BMI categories)
X_val_sample_scaled = scaler.transform(X_val_sample.drop(columns=['BMI_category_Normal', 'BMI_category_Obese', 'BMI_category_Overweight', 'BMI_category_Underweight']))
# Since BMI categories are already one-hot encoded, no need for further encoding
X_val_sample_final = pd.concat([pd.DataFrame(X_val_sample_scaled, columns=X_val_sample.columns.difference(['BMI_category_Normal', 'BMI_category_Obese', 'BMI_category_Overweight', 'BMI_category_Underweight'])),
                                X_val_sample[['BMI_category_Normal', 'BMI_category_Obese', 'BMI_category_Overweight', 'BMI_category_Underweight']]], axis=1)
# Align columns to the same order as in the training data
X_val_sample_final = X_val_sample_final[X_train.columns]
# Make predictions using both models
knn_predictions = knn.predict(X_val_sample_final)
dt_predictions = dt.predict(X_val_sample_final)
# Print the predictions and actual values
print("Predictions from KNN model:", knn_predictions)
print("Predictions from Decision Tree model:", dt_predictions)
print("Actual values:", y_val_sample.values)

Predictions from KNN model: [0 0 0 0 0]
Predictions from Decision Tree model: [0 0 0 0 0]
Actual values: [[0]
 [0]
 [1]
 [1]
 [0]]
